<a href="https://colab.research.google.com/github/Glitch0000/ResNet_Implementaion/blob/main/ResNet_Implementaion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Implementing ResNet

This is a simple example on using ResNet on CIFAR10 dataset.
[Residual Networks](https://arxiv.org/abs/1512.03385) make use of skip connections to make deep models easier to train. 
- There are branches as well as many repeating blocks of layers in this type of network. 
- You can define a model class to help organize this more complex code, and to make it easier to re-use your code when building the model.
- As before, you will inherit from the [Model class](https://keras.io/api/models/model/) so that you can make use of the other built-in methods that Keras provides.

## Imports

In [37]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Layer

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw

## Implement Model subclasses

We first implement the Identity Block which contains the skip connections (i.e. the `add()` operation below. This will also inherit the Model class and implement the `__init__()` and `call()` methods.

In [ ]:
class IdentityBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(IdentityBlock, self).__init__(name='')

        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()

        self.act = tf.keras.layers.Activation('relu')
        self.add = tf.keras.layers.Add()
    
    def call(self, input_tensor):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = self.act(x)

        x = self.conv2(x)
        x = self.bn2(x)

        x = self.add([x, input_tensor])
        x = self.act(x)
        return x

From there, we can build the rest of the ResNet model. 
- We will call your `IdentityBlock` class two times below and that takes care of inserting those blocks of layers into this network.

In [ ]:
class ResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        self.conv = tf.keras.layers.Conv2D(64, 7, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()
        self.act = tf.keras.layers.Activation('relu')
        self.max_pool = tf.keras.layers.MaxPool2D((3, 3))

        # Use the Identity blocks that we just defined
        self.id1a = IdentityBlock(64, 3)
        self.id1b = IdentityBlock(64, 3)

        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.act(x)
        x = self.max_pool(x)

        # insert the identity blocks in the middle of the network
        x = self.id1a(x)
        x = self.id1b(x)

        x = self.global_pool(x)
        return self.classifier(x)

## Training the Model

As mentioned before, inheriting the Model class allows us to make use of the other APIs that Keras provides, such as:
- training
- serialization
- evaluation

We can instantiate a Resnet object and train it as usual like below:



In [35]:
# utility function to normalize the images and return (image, label) pairs.
def preprocess(features):
    return tf.cast(features['image'], tf.float32) / 255., features['label']

# create a ResNet instance with 10 output units for CIfAR10 dataset
resnet = ResNet(10)
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# load and preprocess the dataset
#dataset = tfds.load('Covid19-dataset')
dataset = tfds.load('cifar10', split=tfds.Split.TRAIN)
#dataset = tf.data.Dataset('Covid19-dataset')
dataset = dataset.map(preprocess).batch(32)

# train the model
resnet.fit(dataset, epochs=25)

Epoch 1/25
1563/1563 [==============================] - 12s 7ms/step - loss: 1.5920 - accuracy: 0.4256
Epoch 2/25
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0803 - accuracy: 0.6198
Epoch 3/25
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9133 - accuracy: 0.6769
Epoch 4/25
1563/1563 [==============================] - 8s 5ms/step - loss: 0.7963 - accuracy: 0.7217
Epoch 5/25
1563/1563 [==============================] - 8s 5ms/step - loss: 0.7056 - accuracy: 0.7530
Epoch 6/25
1563/1563 [==============================] - 8s 5ms/step - loss: 0.6371 - accuracy: 0.7785
Epoch 7/25
1563/1563 [==============================] - 8s 5ms/step - loss: 0.5802 - accuracy: 0.7986
Epoch 8/25
1563/1563 [==============================] - 7s 5ms/step - loss: 0.5314 - accuracy: 0.8182
Epoch 9/25
1563/1563 [==============================] - 8s 5ms/step - loss: 0.4877 - accuracy: 0.8328
Epoch 10/25
1563/1563 [==============================] - 8s 5ms/step - loss: 0.44